Imports

In [97]:
import cv2
import numpy as np
import os

In [98]:
show_steps = False

File Names/Dirs

In [99]:
file_dir = "data"
file_name = "PennAir 2024 App Dynamic.mp4"
file = os.path.join(file_dir, file_name)

In [100]:
output_dir = os.path.join("output", "brightness")
output_name = f"annotated_{file_name}"
output = os.path.join(output_dir, output_name)

Load Video

In [101]:
vid = cv2.VideoCapture(file)

In [102]:
# Video properties
fps = vid.get(cv2.CAP_PROP_FPS)
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video writer to save output video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_vid = cv2.VideoWriter(output, fourcc, fps, (width, height))

In [103]:
# Loop through video frames
while True:
    ret, frame = vid.read()

    # Break if no frame is returned
    if not ret:
        break
    
    # Convert to Hue Saturation Value format
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Extract only Value (Brightness)
    val = hsv[:,:,2]

    # Set brightness threshold x
    x = 140
    _, bin_val = cv2.threshold(val, x, 255, cv2.THRESH_BINARY)

    # Clean up w/ morphological operations
    smoothing_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    # Remove noise
    no_noise = cv2.morphologyEx(bin_val, cv2.MORPH_OPEN, smoothing_kernel)

    # Fill gaps + smooth edges
    fill_gaps = cv2.morphologyEx(no_noise, cv2.MORPH_CLOSE, smoothing_kernel)

    # Smooth edges again
    smoothed = cv2.medianBlur(fill_gaps, 5)

    contours, _ = cv2.findContours(smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Remove noise
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        # How much noise to filter
        if area > 500:
            filtered_contours.append(contour)
    
    # Calculate centers of contours
    centers = []
    for contour in filtered_contours:
        # Calculate moments
        M = cv2.moments(contour)
        if M["m00"] != 0: 
            cx = int(M["m10"] / M["m00"])  # x coord
            cy = int(M["m01"] / M["m00"])  # y coord
            centers.append((cx, cy))

    # Copy original image to draw on
    result_frame = frame.copy()

    # Draw contours
    cv2.drawContours(result_frame, filtered_contours, -1, (0, 255, 0), 2)

    for i, (contour, center) in enumerate(zip(filtered_contours, centers)):
        cx, cy = center
        
        # Get bounding box for text placement
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw center
        cv2.circle(result_frame, center, 5, (255, 255, 255), -1)
        
        # Place text
        text = f"({cx},{cy})"
        text_x = cx - 80
        text_y = y + h + 40
        cv2.putText(result_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    
    if show_steps:
        # Display results
        cv2.imshow('Video Results', result_frame)
        if cv2.waitKey(1) & 0xFF == 13:
            break

vid.release()
out_vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

KeyboardInterrupt: 